In [ ]:
import folium
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [127]:
Adr = pd.read_csv('Dataset/Air_Quality_20250217.csv')
geo = pd.read_csv('Dataset/GeoLookup.csv', usecols=['GeoType', 'GeoID', 'Lat', 'Long'])
Adr.drop('Message', axis=1, inplace=True)

In [18]:
#COLUMN INFORMATION (Adr)
#unique_id: Unique record identifier
#indicator_id: Idientifier of the type of measured value across time and space
#name: Name of the indicator
#measure: How the indicator is measured
#measure_info: Information (units) about the measure
#GeoType: Geography type
#GeoID: Identifier of the neighborhood geographic area, used for joining to mapping geography files to make thematic maps
#geo_place_name: Neighborhood name
#time_period: Description of the time that the data applies to
#start_date: Date value for the start of the time period
#data_value: The actual data value for this indicator, measure, place, and time
#message: notes that apply to the data value

#COLUMN INFORMATION (geo)
#GeoType: Geography type
#GeoID: Identifier of the neighborhood geographic area, used for joining to mapping geography files to make thematic maps
#Lat: latitude
#Long: longitude

In [128]:
# Limpieza
Adr.rename(columns={'Geo Type Name': 'GeoType', 'Geo Join ID': 'GeoID'}, inplace=True)
Adr.dropna(inplace=True)
geo.dropna(inplace=True)
Adr['GeoID'] = Adr['GeoID'].astype('int64')
Adr.drop(columns=['Unique ID', 'Indicator ID', 'Measure', 'Start_Date'], inplace=True)


In [129]:
# Filtrar solo las indicadores que se usarán y el periodo de tiempo anual
Adr = Adr[Adr['Time Period'].isin(['2005-2007', '2009-2011', '2012-2014', '2015-2017', '2017-2019'])].reset_index(drop=True)

# Merge para obtener las coordenadas
Adr = Adr.merge(geo, how='left', on=['GeoType', 'GeoID'])
Adr.dropna(inplace=True)
Adr[Adr.select_dtypes(include='object').columns] = Adr.select_dtypes(include='object').apply(lambda x: x.astype('category'))

def name_change(row):
    if 'children' in row['Measure Info']:
        return row['Name'] + ' (children)'
    elif 'adult' in row['Measure Info']:
        return row['Name'] + ' (adult)'
    else:
        return row['Name']
    
Adr['Name'] = Adr.apply(name_change, axis = 1)
Adr

,Name,Measure Info,GeoType,GeoID,Geo Place Name,Time Period,Data Value,Lat,Long
0,Asthma emergency department visits due to PM2....,"per 100,000 children",UHF42,208,Canarsie - Flatlands,2009-2011,81.7,40.6203,-73.9037
1,Asthma emergency department visits due to PM2....,"per 100,000 children",UHF42,407,Southwest Queens,2009-2011,80.0,40.6795,-73.8355
2,Asthma emergency department visits due to PM2....,"per 100,000 children",UHF42,207,East Flatbush - Flatbush,2009-2011,115.8,40.6464,-73.9484
3,Asthma emergency department visits due to PM2....,"per 100,000 children",UHF42,408,Jamaica,2015-2017,69.6,40.6960,-73.7831
4,Asthma emergency department visits due to PM2....,"per 100,000 children",UHF42,407,Southwest Queens,2015-2017,43.0,40.6795,-73.8355
...,...,...,...,...,...,...,...,...,...
2395,Respiratory hospitalizations due to PM2.5 (age...,"per 100,000 adults",UHF42,408,Jamaica,2012-2014,9.8,40.6960,-73.7831
2396,Cardiac and respiratory deaths due to Ozone,"per 100,000",UHF42,307,Gramercy Park - Murray Hill,2009-2011,3.6,40.7489,-73.9753
2397,Cardiac and respiratory deaths due to Ozone,"per 100,000",UHF42,107,Hunts Point - Mott Haven,2009-2011,3.1,40.8131,-73.8988
2398,Deaths due to PM2.5 (adult),"per 100,000 adults",UHF42,308,Greenwich Village - SoHo,2009-2011,41.0,40.7265,-74.0042


In [130]:
period = widgets.Dropdown(
    options = sorted(Adr['Time Period'].unique()),
    description = 'Select time period',
    disabled = False
)

indicator = widgets.Dropdown(
    options = sorted(Adr['Name'].unique()), #(*) cambiar a options = []
    description = 'Select indicator',
    disabled = False
)

# (*) Para hacer un doble filtro dinamico, si cada periodo tienen un indicador diferente 

#def update_indicators(*args):
#    selected_period = period.value
#    available_indicators = Adr[Adr['Time Period'] == selected_period]['Name'].unique()
#    indicator.options = sorted(available_indicators)

#period.observe(update_indicators, 'value')

def tool_t (row):
    return f"{row['Geo Place Name']} <br> Valor: {row['Data Value']} {row['Measure Info']}"

In [131]:
def AQI_period (pr, ind):
    map = folium.Map(location=[40.7122, -73.9793], zoom_start=11, tiles="CartoDB dark_matter")

    for _, row in Adr[(Adr['Time Period'] == pr) & (Adr['Name'] == ind)].iterrows():
        tooltip_text = tool_t(row)

        # Color según Data Value
        if 'Asthma emergency' in row['Name']:
                if row['Data Value'] <= 50:
                        color = '#009966'
                elif row['Data Value'] > 50 and row['Data Value'] <= 100:
                        color = '#ffde33'
                elif row['Data Value'] > 100 and row['Data Value'] <= 150:
                        color = '#ff9933'
                elif row['Data Value'] > 150 and row['Data Value'] <= 200:
                        color = 'rgb(204, 0, 51)'
                elif row['Data Value'] > 200 and row['Data Value'] <= 300:
                        color = '#660099'
                elif row['Data Value'] > 300:
                        color = '#7e0023'
        
        if 'hospitalization' in row['Name']:
                if row['Data Value'] <= 15:
                      color = '#009966'
                elif row['Data Value'] > 15 and row['Data Value'] <= 30:
                        color = '#ffde33'
                elif row['Data Value'] > 30 and row['Data Value'] <= 45:
                        color = '#ff9933'
                elif row['Data Value'] > 45 and row['Data Value'] <= 60:
                        color = 'rgb(204, 0, 51)'
                elif row['Data Value'] > 60 and row['Data Value'] <= 70:
                        color = '#660099'
                elif row['Data Value'] > 70:
                        color = '#7e0023'
        
        if 'deaths' in row['Name'] or 'Deaths' in row['Name']:
                if row['Data Value'] <= 5:
                      color = '#009966'
                elif row['Data Value'] > 5 and row['Data Value'] <= 15:
                        color = '#ffde33'
                elif row['Data Value'] > 15 and row['Data Value'] <= 30:
                        color = '#ff9933'
                elif row['Data Value'] > 30 and row['Data Value'] <= 50:
                        color = 'rgb(204, 0, 51)'
                elif row['Data Value'] > 50 and row['Data Value'] <= 70:
                        color = '#660099'
                elif row['Data Value'] > 70:
                        color = '#7e0023'
                        
        folium.CircleMarker(
            location=[row['Lat'], row['Long']],
                radius=6,
                color= color,
                stroke=False,
                fill=True,
                fill_opacity=1,
                tooltip= tooltip_text,
                popup= folium.Popup(tooltip_text, max_width=360),
                opacity=1,
            ).add_to(map)
    
    display(map)

In [126]:
widgets.interactive(AQI_period, pr = period, ind = indicator)

interactive(children=(Dropdown(description='Select time period', options=('2005-2007', '2009-2011', '2012-2014…